In [ ]:
%%capture
# export
import dataclasses

from fastai2.basics import *
from transformers import PreTrainedTokenizer, AutoTokenizer

from fastai2_utils.pytorch.transformer import *
from fastai_transformers_utils.generated_lm import GeneratedLM, GenerateArgs
from fastai_transformers_utils.tokenizers import GPT2DecoderTokenizer

In [ ]:
# default_exp models.tran2tran

# Models Tran2Tran
> 

In [ ]:
bs = 3
enc_seq_len = 50
dec_seq_len = 40

## ModelArgs

In [ ]:
enc_max_pos_id = 100
enc_vocab_size = 21128
enc_pad_id = 0

dec_max_pos_id = 100
dec_vocab_size = 50259
dec_pad_id = 50258

embeded_size = 512
num_head = 8
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048
drop_p = 0.1

## WPEmbed

In [ ]:
# export
# https://github.com/kaushalshetty/Positional-Encoding/blob/master/encoder.py
def _position_encoding_init(n_position, emb_dim):
    ''' Init the sinusoid position encoding table '''

    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / emb_dim) for j in range(emb_dim)]
        if pos != 0 else np.zeros(emb_dim) for pos in range(n_position)])
    

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # apply sin on 0th,2nd,4th...emb_dim
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # apply cos on 1st,3rd,5th...emb_dim
    return torch.from_numpy(position_enc).type(torch.FloatTensor)

In [ ]:
# export
class WPEmbed(nn.Module):
    '''
        returns position_embedding + word_embedding
    '''
    def __init__(self, vocab_size, embeded_size, max_pos_id, pad_id):
        super().__init__()
        self.word_embedding = nn.Embedding(vocab_size, embeded_size, padding_idx=pad_id)
        self.position_embedding = nn.Embedding(max_pos_id, embeded_size)
        self.position_embedding.weight.data = _position_encoding_init(max_pos_id, embeded_size)
        
        self.embeded_size = embeded_size
        
    def forward(self, input_ids):
        '''
            input_ids: (b, seq_len)
            returns: (b, seq_len, embed_size)
        '''
        device = input_ids.device
        bs, seq_len = input_ids.shape
        
        position_ids = torch.arange(seq_len, dtype=torch.long, device=device).expand(input_ids.shape) # (b, seq_len)
        position_embedding = self.position_embedding(position_ids) # (b, seq_len, embeded_size)
        
        word_embedding = self.word_embedding(input_ids) # (b, seq_len, embeded_size)
        
        # This scale factor is very important for your model to converge faster!!!!!!!!!!
        scale = torch.sqrt(torch.tensor(self.embeded_size, device=device).float())
        
        return word_embedding*scale + position_embedding  # (b, seq_len, embeded_size)

In [ ]:
enc_emb = WPEmbed(enc_vocab_size, embeded_size, enc_max_pos_id, enc_pad_id)
input_ids = torch.randint(0, enc_vocab_size, (bs, enc_seq_len)) # (bs, enc_seq_len)
embed = enc_emb(input_ids) # (bs, enc_seq_len, embeded_size)
test_eq(embed.shape, (bs, enc_seq_len, embeded_size))

## TranEncoder

In [ ]:
# export
class TranEncoder(nn.Module):
    '''
        The encoder of transformer architecture.
        WPEmbed -> TransformerEncoder.
    '''
    def __init__(
        self,
        vocab_size, embeded_size, max_pos_id, pad_id, # for WPEmbed
        num_head=8, num_encoder_layers=6, dim_feedforward=2048, drop_p=0.1, # for TransformerEncoder
    ):
        super().__init__()
        self.embedding = WPEmbed(vocab_size, embeded_size, max_pos_id, pad_id)
        
        encoder_layer = nn.TransformerEncoderLayer(embeded_size, num_head, dim_feedforward, drop_p)
        encoder_norm = nn.LayerNorm(embeded_size)
        self.encoder = BatchFirstTransformerEncoder(encoder_layer, num_encoder_layers)
        
        self.pad_id = pad_id
        
        self._init_weights()
    
    def forward(self, src_input_ids, src_key_padding_mask):
        '''
            src_input_ids: (bs, enc_seq_len)
            src_key_padding_mask: Booling mask that True is padding position with shape (bs, enc_seq_len)
            returns: 
                memory: (bs, enc_seq_len, embeded_size)
        '''
        embed = self.embedding(src_input_ids) # (bs, enc_seq_len)
        memory = self.encoder(embed, src_key_padding_mask=src_key_padding_mask) # (bs, enc_seq_len, embeded_size)
        return memory # (bs, enc_seq_len, embeded_size)
    
    def _init_weights(self):
        for p in self.encoder.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

In [ ]:
encoder = TranEncoder(
    enc_vocab_size, embeded_size, enc_max_pos_id, enc_pad_id, 
    num_head, num_encoder_layers, dim_feedforward, drop_p
)
src_input_ids = torch.randint(0, enc_vocab_size, (bs, enc_seq_len)) # (bs, enc_seq_len)
src_key_padding_mask = torch.zeros((bs, enc_seq_len)).bool() # (bs, enc_seq_len)
memory = encoder(src_input_ids, src_key_padding_mask) # (bs, enc_seq_len, embeded_size)
test_eq(memory.shape, (bs, enc_seq_len, embeded_size))

## TranDecoder

In [ ]:
# export
class TranDecoder(nn.Module):
    '''
        The decoder of transformer architecture.
        WPEmbed -> TransformerDecoder -> Linear Classifier
    '''
    def __init__(
        self,
        vocab_size, embeded_size, max_pos_id, pad_id, # for WPEmbed and Classifier
        num_head=8, num_decoder_layers=6, dim_feedforward=2048, drop_p=0.1, # for TransformerDecoder
    ):
        super().__init__()
        self.embedding = WPEmbed(vocab_size, embeded_size, max_pos_id, pad_id)
        
        decoder_layer = nn.TransformerDecoderLayer(embeded_size, num_head, dim_feedforward, drop_p)
        decoder_norm = nn.LayerNorm(embeded_size)
        self.decoder = BatchFirstTransformerDecoder(decoder_layer, num_decoder_layers)
        
        self.classifier = nn.Linear(embeded_size, vocab_size) # Language Model
        
        self.pad_id = pad_id
    
        self._init_weights()

    def forward(self, tgt_input_ids, memory, memory_key_padding_mask):
        '''
            tgt_input_ids: (bs, dec_seq_len)
            memory: (bs, enc_seq_len, embeded_size)
            memory_key_padding_mask: (bs, enc_seq_len)
            memory_mask: (dec_seq_len, enc_seq_len)
            returns: (bs, dec_seq_len, dec_vocab_size), None
        '''
        seq_len = tgt_input_ids.shape[1]
        embed = self.embedding(tgt_input_ids) # (bs, dec_seq_len)
        tgt_key_padding_mask = gen_key_padding_mask(tgt_input_ids, self.pad_id) # (bs, dec_seq_len)
        tgt_mask = gen_lm_mask(seq_len, tgt_input_ids.device) #(dec_seq_len, dec_seq_len)
        
        output = self.decoder(embed, memory, tgt_mask=tgt_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask) # (bs, dec_seq_len, embeded_size)
        
        output = self.classifier(output) # (bs, dec_seq_len, dec_vocab_size)
        return output, None # (bs, dec_seq_len, dec_vocab_size)
    
    def _init_weights(self):
        for p in self.decoder.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        for p in self.classifier.parameters():
            if p.dim() > 1:
                nn.init.kaiming_uniform_(p)

In [ ]:
decoder = TranDecoder(
    dec_vocab_size, embeded_size, dec_max_pos_id, dec_pad_id, 
    num_head, num_decoder_layers, dim_feedforward, drop_p
)

tgt_input_ids = torch.randint(0, dec_vocab_size, (bs, dec_seq_len)) # (bs, dec_seq_len)
memory = torch.randn((bs, enc_seq_len, embeded_size)) # (bs, enc_seq_len, embeded_size)
memory_key_padding_mask = torch.zeros((bs, enc_seq_len)).bool() # (bs, enc_seq_len)

output = decoder(tgt_input_ids, memory, memory_key_padding_mask=memory_key_padding_mask) # (bs, dec_seq_len, dec_vocab_size)
test_eq(output[0].shape, (bs, dec_seq_len, dec_vocab_size))

## Tran2Tran

In [ ]:
# export
class Tran2Tran(nn.Module):
    '''
        Seq2Seq architecture. Encoder and decoder are transformer.
    '''
    def __init__(
        self, 
        encoder: TranEncoder, decoder: TranDecoder, 
        enc_pad_id, # for src_key_padding_mask and memory_key_padding_mask
    ):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

        self.enc_pad_id = enc_pad_id
        
    def forward(self, src_input_ids, tgt_input_ids):
        '''
           src_input_ids: (bs, enc_seq_len)
           tgt_input_ids: (bs, dec_seq_len)
           returns: (bs, dec_seq_len, dec_vocab_size)
        '''
        
        src_key_padding_mask = gen_key_padding_mask(src_input_ids, self.enc_pad_id) # (bs, enc_seq_len)
        memory = self.encoder(src_input_ids, src_key_padding_mask=src_key_padding_mask) # (bs, enc_seq_len, embeded_size)
        output, _ = self.decoder(tgt_input_ids, memory, memory_key_padding_mask=src_key_padding_mask) # (bs, dec_seq_len, embeded_size)
        return output

In [ ]:
# encoder = TranEncoder(enc_vocab_size, embeded_size, enc_max_pos_id, enc_pad_id)
# decoder = TranDecoder(dec_vocab_size, embeded_size, dec_max_pos_id, dec_pad_id)
tran2tran = Tran2Tran(encoder, decoder, enc_pad_id)

src_input_ids = torch.randint(0, enc_vocab_size, (bs, enc_seq_len)) # (bs, enc_seq_len)
tgt_input_ids = torch.randint(0, dec_vocab_size, (bs, dec_seq_len)) # (bs, dec_seq_len)
output = tran2tran(src_input_ids, tgt_input_ids) # (bs, dec_seq_len, dec_vocab_size)
test_eq(output.shape, (bs, dec_seq_len, dec_vocab_size))

## GeneratedTran2Tran

In [ ]:
# export
class GeneratedTran2Tran():
    def __init__(
        self, 
        seq2seq: Tran2Tran, 
        enc_tokenizer: PreTrainedTokenizer,
        dec_tokenizer: PreTrainedTokenizer,
    ):
        self.seq2seq = seq2seq
        self.enc_tokenizer = enc_tokenizer
        self.dec_tokenizer = dec_tokenizer
        self.generatedLM = GeneratedLM(seq2seq.decoder, len(dec_tokenizer), dec_tokenizer.pad_token_id, [dec_tokenizer.eos_token_id], support_past=False)

In [ ]:
enc_tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm-ext')
dec_tokenizer = GPT2DecoderTokenizer.from_pretrained('distilgpt2')

### generate_from_ids

In [ ]:
# export
@patch
@torch.no_grad()
def generate_from_ids(self: GeneratedTran2Tran, src_input_ids, generate_args: GenerateArgs):
    ''' src_input_ids: (bs, enc_seq_len), returns: (bs, max_length)'''
    self.seq2seq.eval()
    
    device = src_input_ids.device
    bs = src_input_ids.shape[0]
    tgt_input_ids = torch.zeros((bs, 1), dtype=torch.long, device=device).fill_(self.dec_tokenizer.bos_token_id) # (bs, 1)

    src_key_padding_mask = gen_key_padding_mask(src_input_ids, self.enc_tokenizer.pad_token_id) # (bs, enc_seq_len)
    memory = self.seq2seq.encoder(src_input_ids, src_key_padding_mask) # (bs, enc_seq_len, embeded_size)
    model_otherargs = self.generatedLM.build_model_otherargs_for_beam([memory, src_key_padding_mask], generate_args.num_beams)

    result = self.generatedLM.generate(tgt_input_ids, generate_args, [model_otherargs[0]], dict(memory_key_padding_mask=model_otherargs[1]))

    return result

In [ ]:
generated_tran2tran = GeneratedTran2Tran(tran2tran, enc_tokenizer, dec_tokenizer)

generate_args = GenerateArgs(max_length=10, num_beams=2)
src_input_ids = torch.randint(0, enc_vocab_size, (bs, enc_seq_len)) # (bs, enc_seq_len)
result = generated_tran2tran.generate_from_ids(src_input_ids, generate_args)
test_eq(result.shape, (bs, 10))

## Export -

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 02_data.tatoeba.ipynb.
Converted 03a_models.patch.ipynb.
Converted 03c_models.bert2gpt2.ipynb.
Converted 03c_models.gru2gru.ipynb.
Converted 03c_models.tran2tran.ipynb.
Converted 04_metrics.ipynb.
Converted index.ipynb.
